In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import glob
from time import time
import keras.backend as K
from keras.initializers import RandomNormal

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
continueTraining = True
base_path = './data/'

In [ ]:
class MinibatchDiscrimination(tf.keras.layers.Layer):
    def __init__(self, n_kernels, kernel_dim, **kwargs):
        super(MinibatchDiscrimination, self).__init__(**kwargs)
        self.n_kernels = n_kernels
        self.kernel_dim = kernel_dim

    def build(self, input_shape):
        self.C = self.add_weight(name='minibatch_discrimination_matrix',
                                  shape=(input_shape[1], self.n_kernels, self.kernel_dim),
                                  initializer='uniform',
                                  trainable=True)
        # self.bias = self.add_weight(name='minibatch_discrimination_bias',
        #                              shape=(self.n_kernels, self.kernel_dim,),
        #                              initializer='zeros',
        #                              trainable=True)

    def call(self, inputs, **kwargs):
        activation = K.dot(inputs, self.C)
        abs_dif = K.abs(K.expand_dims(activation, 3) - K.expand_dims(K.permute_dimensions(activation, [1, 2, 0]), 0))
        minibatch_features = K.sum(K.exp(-K.sum(abs_dif, axis=2) / self.kernel_dim), axis=2)
        return K.concatenate([inputs, minibatch_features], axis=1)


In [ ]:
# The discriminator model takes an image as input and returns a probability
# indicating whether the image is real or fake
def make_discriminator_model():
    model = tf.keras.Sequential()

    # Prevent overfitting
    model.add(layers.GaussianNoise(0.2, input_shape = [256, 256, 3]))

    model.add(layers.Conv2D(32, (5, 5), strides=(2, 2), padding='same'))
    layers.BatchNormalization(momentum=0.8)
    model.add(layers.LeakyReLU())
    # model.add(layers.GaussianNoise(0.1))
    model.add(layers.Dropout(0.4))

    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
    layers.BatchNormalization(momentum=0.8)
    model.add(layers.LeakyReLU())
    # model.add(layers.GaussianNoise(0.1))
    model.add(layers.Dropout(0.4))
  
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    layers.BatchNormalization(momentum=0.8)
    model.add(layers.LeakyReLU())
    # model.add(layers.GaussianNoise(0.1))
    model.add(layers.Dropout(0.4))

    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    layers.BatchNormalization(momentum=0.8)
    model.add(layers.LeakyReLU())
    # model.add(layers.GaussianNoise(0.1))
    model.add(layers.Dropout(0.4))
  
    model.add(layers.Flatten())
    model.add(layers.Dense(256, use_bias=False))
    model.add(MinibatchDiscrimination(256, 3))
    model.add(layers.Dense(1))
  
    return model

In [ ]:
# The generator model takes a noise vector as input and returns an image
def make_generator_model():
    init = RandomNormal(mean=0.0, stddev=0.02, seed=42)
    model = tf.keras.Sequential()
    model.add(layers.Dense(8 * 8 * 256, use_bias=False, input_shape=(100,), kernel_initializer = init))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((8, 8, 256)))
    

    # 8x8x256
    model.add(layers.Conv2D(256, kernel_size = 3, padding = "same", kernel_initializer = init))
    assert model.output_shape == (None, 8, 8, 256) # Note: None is the batch size
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.LeakyReLU())

    # 16x16x128
    model.add(layers.UpSampling2D())
    model.add(layers.Conv2D(128, kernel_size = 3, padding = "same", kernel_initializer = init))
    assert model.output_shape == (None, 16, 16, 128)
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.LeakyReLU())
    
    # 32x32x64
    model.add(layers.UpSampling2D())
    model.add(layers.Conv2D(64, kernel_size = 3, padding = "same", kernel_initializer = init))
    assert model.output_shape == (None, 32, 32, 64)
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.LeakyReLU())

    # 64x64x32
    model.add(layers.UpSampling2D())
    model.add(layers.Conv2D(32, kernel_size = 3, padding = "same", kernel_initializer = init))
    assert model.output_shape == (None, 64, 64, 32)
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.LeakyReLU())

    # 128x128x16
    model.add(layers.UpSampling2D())
    model.add(layers.Conv2D(16, kernel_size = 3, padding = "same", kernel_initializer = init))
    assert model.output_shape == (None, 128, 128, 16)
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.LeakyReLU())

    # 256x256x8
    model.add(layers.UpSampling2D())
    model.add(layers.Conv2D(8, kernel_size = 3, padding = "same", kernel_initializer = init))
    assert model.output_shape == (None, 256, 256, 8)
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(3, kernel_size = 3, padding = "same", kernel_initializer = init))
    model.add(layers.Activation("tanh"))

    return model

In [ ]:
# Create the discriminator and generator models
if continueTraining:
    discriminator = models.load_model(base_path + 'discriminator')
    generator = models.load_model(base_path + 'generator')
else:
    discriminator = make_discriminator_model()
    generator = make_generator_model()

# The generator takes noise as input and generates images
noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

# Use Binary Crossentropy loss for both the generator and discriminator
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
accuracy_metric = tf.keras.metrics.BinaryAccuracy()

In [ ]:
def gradient_penalty(images, generated_images):
    alpha = tf.random.uniform([BATCH_SIZE, 1, 1, 1], 0.0, 1.0)
    interpolated_images = alpha * images + (1 - alpha) * generated_images

    with tf.GradientTape() as tape:
        tape.watch(interpolated_images)
        d_interpolated = discriminator(interpolated_images, training=True)

    gradients = tape.gradient(d_interpolated, interpolated_images)
    slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis=[1, 2, 3]))
    gradient_penalty = tf.reduce_mean(tf.square(slopes - 1.0))
    return gradient_penalty

In [ ]:
def wasserstein_loss(real_output, fake_output, gp = 0):
    return tf.reduce_mean(fake_output) - tf.reduce_mean(real_output) + 10 * gp

def discriminator_loss(real_output, fake_output, gp = 0):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss + gp * 10
    # return tf.reduce_mean(fake_output) - tf.reduce_mean(real_output)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)


In [ ]:
def get_accuracy(fake_output):
    accuracy_metric.reset_state()
    accuracy_metric.update_state(tf.ones_like(fake_output), tf.nn.sigmoid(fake_output))
    return accuracy_metric.result()

In [ ]:
# Define the optimizers for both generator and discriminator
generator_optimizer = tf.keras.optimizers.Adam(4e-5, 0.5)
# discriminator_optimizer = tf.keras.optimizers.Adam(4e-5, 0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(4e-5, 0.5)

# Define the training loop for the GAN
EPOCHS = 1000
noise_dim = 100
num_examples_to_generate = 4
BATCH_SIZE = 32

# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
  
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
      
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gp = gradient_penalty(images, generated_images)
    
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output, gp)

    accuracy = get_accuracy(fake_output);
        
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    return gen_loss, disc_loss, accuracy

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    # make sure the training parameter is set to False because we
    # don't want to train the batchnorm layer when doing inference.
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(2, 2))
    
    for i in range(predictions.shape[0]):
        plt.subplot(2, 2, i+1)
        plt.imshow(predictions[i, :, :, :] * 0.5 + 0.5)
        plt.axis('off')
        
    plt.savefig(base_path + 'genned/image_at_epoch_{:04d}.png'.format(epoch), dpi=320)
    plt.close('all')

In [ ]:
def train(dataset, epochs):
    start = time()
    for epoch in range(epochs):
        print("Epoch: ", epoch)
        for image_batch in dataset:
            gen_loss, disc_loss, accuracy = train_step(image_batch)

        accuracy = accuracy.numpy()
        
        print("Generator loss: ", gen_loss)
        print("Discriminator loss: ", disc_loss)
        print("Accuracy: ", accuracy)
        print("--- %s seconds ---" % (time() - start))
      
        # Produce images for the GIF as we go
        # clear_output(wait=False)
        if epoch % 4 == 0:
          generate_and_save_images(generator,
                                    epoch + 1,
                                    seed)
        if epoch % 50 == 0 and epoch != 0:
            generator.save(base_path + 'generator')
            discriminator.save(base_path + 'discriminator')
        
    # Generate after the final epoch
    # clear_output(wait=True)
        # Generate after the final epoch
    generate_and_save_images(generator,
                             epochs,
                             seed)

In [ ]:
def load_image(image_file):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_jpeg(image)
    image = tf.image.resize(image, [256, 256])
    image = (image - 127.5) / 127.5
    return image

def load_dataset(folder_path):
    all_images = glob.glob(folder_path + '/*.jpg')
    dataset = tf.data.Dataset.from_tensor_slices(all_images)
    dataset = dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

In [ ]:
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
# tf.config.experimental_connect_to_cluster(resolver)
# # This is the TPU initialization code that has to be at the beginning.
# tf.tpu.experimental.initialize_tpu_system(resolver)
# print("All devices: ", tf.config.list_logical_devices('TPU'))

In [ ]:
train_dataset = load_dataset(base_path + 'compressed')
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

# Train the GAN
train(train_dataset, EPOCHS)

Epoch:  0
Generator loss:  tf.Tensor(2.8051524, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.35996988, shape=(), dtype=float32)
Accuracy:  0.03125
--- 62.43106150627136 seconds ---
Epoch:  1
Generator loss:  tf.Tensor(6.1607594, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.17342399, shape=(), dtype=float32)
Accuracy:  0.0
--- 75.75688481330872 seconds ---
Epoch:  2
Generator loss:  tf.Tensor(5.341772, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.129733, shape=(), dtype=float32)
Accuracy:  0.0
--- 86.93947005271912 seconds ---
Epoch:  3
Generator loss:  tf.Tensor(6.408815, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.23841129, shape=(), dtype=float32)
Accuracy:  0.0
--- 98.2360532283783 seconds ---
Epoch:  4
Generator loss:  tf.Tensor(5.893613, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.18870166, shape=(), dtype=float32)
Accuracy:  0.0
--- 109.65928077697754 seconds ---
Epoch:  5
Generator loss:  tf.Tensor(4.5886855, sha

Generator loss:  tf.Tensor(6.358908, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.21085891, shape=(), dtype=float32)
Accuracy:  0.0
--- 649.6194798946381 seconds ---


Epoch:  51
Generator loss:  tf.Tensor(3.629484, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.28929192, shape=(), dtype=float32)
Accuracy:  0.0625
--- 666.897097826004 seconds ---
Epoch:  52
Generator loss:  tf.Tensor(5.0832353, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.17048779, shape=(), dtype=float32)
Accuracy:  0.0
--- 678.4017143249512 seconds ---
Epoch:  53
Generator loss:  tf.Tensor(5.5696692, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.13109006, shape=(), dtype=float32)
Accuracy:  0.0
--- 691.0545873641968 seconds ---
Epoch:  54
Generator loss:  tf.Tensor(6.094549, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.18143815, shape=(), dtype=float32)
Accuracy:  0.0
--- 702.5775220394135 seconds ---
Epoch:  55
Generator loss:  tf.Tensor(4.318159, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.14437877, shape=(), dtype=float32)
Accuracy:  0.0
--- 714.0356471538544 seconds ---
Epoch:  56
Generator loss:  tf.Tensor(4.918328

Epoch:  101
Generator loss:  tf.Tensor(5.9137673, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.31477684, shape=(), dtype=float32)
Accuracy:  0.0
--- 1257.6233341693878 seconds ---
Epoch:  102
Generator loss:  tf.Tensor(4.898142, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.3227539, shape=(), dtype=float32)
Accuracy:  0.03125
--- 1269.1094210147858 seconds ---
Epoch:  103
Generator loss:  tf.Tensor(5.1410217, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.32793954, shape=(), dtype=float32)
Accuracy:  0.0
--- 1280.6254467964172 seconds ---
Epoch:  104
Generator loss:  tf.Tensor(6.0266347, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.41564342, shape=(), dtype=float32)
Accuracy:  0.0
--- 1292.1292686462402 seconds ---
Epoch:  105
Generator loss:  tf.Tensor(6.2382855, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.18089983, shape=(), dtype=float32)
Accuracy:  0.0
--- 1304.7292428016663 seconds ---
Epoch:  106
Generator loss:  tf.T

Generator loss:  tf.Tensor(6.073905, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.22272497, shape=(), dtype=float32)
Accuracy:  0.0
--- 1832.1211199760437 seconds ---


...

Generator loss:  tf.Tensor(6.7083416, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.073662296, shape=(), dtype=float32)
Accuracy:  0.0
--- 10031.074781894684 seconds ---


Epoch:  851
Generator loss:  tf.Tensor(6.15431, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.16879663, shape=(), dtype=float32)
Accuracy:  0.0
--- 10047.938520669937 seconds ---
Epoch:  852
Generator loss:  tf.Tensor(5.9335775, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.12843898, shape=(), dtype=float32)
Accuracy:  0.0
--- 10059.475965976715 seconds ---
Epoch:  853
Generator loss:  tf.Tensor(5.798095, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.06981453, shape=(), dtype=float32)
Accuracy:  0.0
--- 10071.26538157463 seconds ---
Epoch:  854
Generator loss:  tf.Tensor(4.7551703, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.12018508, shape=(), dtype=float32)
Accuracy:  0.0
--- 10082.752993106842 seconds ---
Epoch:  855
Generator loss:  tf.Tensor(4.4724116, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.09756391, shape=(), dtype=float32)
Accuracy:  0.0
--- 10094.232053995132 seconds ---
Epoch:  856
Generator loss:  tf.Tensor(

Epoch:  901
Generator loss:  tf.Tensor(6.517454, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.07833587, shape=(), dtype=float32)
Accuracy:  0.0
--- 10628.447930812836 seconds ---
Epoch:  902
Generator loss:  tf.Tensor(5.3990154, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.081764795, shape=(), dtype=float32)
Accuracy:  0.0
--- 10639.892546415329 seconds ---
Epoch:  903
Generator loss:  tf.Tensor(5.969142, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.088010006, shape=(), dtype=float32)
Accuracy:  0.0
--- 10651.373501300812 seconds ---
Epoch:  904
Generator loss:  tf.Tensor(5.463318, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.12736356, shape=(), dtype=float32)
Accuracy:  0.0
--- 10662.841529369354 seconds ---
Epoch:  905
Generator loss:  tf.Tensor(4.6110506, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.11922076, shape=(), dtype=float32)
Accuracy:  0.03125
--- 10674.540569782257 seconds ---
Epoch:  906
Generator loss:  tf.

Generator loss:  tf.Tensor(6.17881, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.10611613, shape=(), dtype=float32)
Accuracy:  0.0
--- 11192.641881465912 seconds ---


Epoch:  951
Generator loss:  tf.Tensor(5.617921, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.1090056, shape=(), dtype=float32)
Accuracy:  0.0
--- 11209.419004678726 seconds ---
Epoch:  952
Generator loss:  tf.Tensor(5.094881, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.05780989, shape=(), dtype=float32)
Accuracy:  0.0
--- 11220.90805387497 seconds ---
Epoch:  953
Generator loss:  tf.Tensor(5.844898, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.07533452, shape=(), dtype=float32)
Accuracy:  0.0
--- 11232.664379119873 seconds ---
Epoch:  954
Generator loss:  tf.Tensor(7.2358074, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.14087096, shape=(), dtype=float32)
Accuracy:  0.0
--- 11244.127228021622 seconds ---
Epoch:  955
Generator loss:  tf.Tensor(5.56658, shape=(), dtype=float32)
Discriminator loss:  tf.Tensor(0.058223985, shape=(), dtype=float32)
Accuracy:  0.0
--- 11255.538375139236 seconds ---
Epoch:  956
Generator loss:  tf.Tensor(6.